In [1]:
import numpy as np
import openpyxl
import ast
from sympy.physics.quantum import TensorProduct
from sympy import Matrix
import numpy as np

In [2]:
path = "C:\\Users\\LENOVO\\OneDrive - National University of Sciences & Technology\\Academia\\FYP\\TSP-4.xlsx"

wb = openpyxl.load_workbook(path)
ws = wb.active

In [8]:
def distance(i):
    distance_matrix =np.array(ast.literal_eval(ws[f'A{i+2}'].value))
    m2 = np.array(ast.literal_eval(ws[f'D{i+2}'].value))
    return distance_matrix

In [14]:
n = 4
beta_para = 1.39957953
theta1 = [1.9026521]
theta2 = [2.22944427]
alpha = [5.56530919]


initial_point = np.array([beta_para for i in range(n*n)]+[theta1[0] for i in range(n*n)]+
                         [theta2[0] for i in range(n*n)]+[alpha[0] for i in range(n*n)])

reference_circuit = QuantumCircuit(n*n)      # Circuit to which data be encoded
variational_circuit = QuantumCircuit(n*n)     # Circuit for


loss = 0
MCRX=RXGate(beta_para).control(6, ctrl_state='000000')

In [ ]:
for K in range(10):
    TSP = TSP_Solver(reference_circuit, variational_circuit, MCRX, n, K, theta1, theta2, alpha, loss)

    def objective_function(params):

        theta1_values = params[16:32]  # Extract first 9 values for theta1
        theta2_values = params[32:48]  # Extract next 9 values for theta2
        alpha = params[48:64]  # Extract alpha
        beta_para = params[:1]  # Extract beta_para

        # Set theta1 and theta2 values
        TSP.theta1 = theta1_values
        TSP.theta2 = theta2_values
        TSP.alpha = alpha

        # Update the beta_para for MCRX gate
        beta_angle = beta_para[0]  # Assuming you want to use the first value from beta_para
        MCRX = RXGate(beta_angle).control(6, ctrl_state='000000')
        TSP.MCRX = MCRX

        # Run the quantum neural network and compute loss
        obj_loss = call_pass_obj_fun(TSP)
        loss += obj_loss
        
        return loss

    def gradient_function(params):

        epsilon = 1e-6  # pert value

        initial_loss = objective_function(params)

        params_plus = params.copy() + epsilon

        loss_plus = objective_function(params_plus)

        gradient = (loss_plus - initial_loss) / epsilon

        return gradient

    # Instantiate Adam optimizer
    adam_optimizer = ADAM(maxiter= 1, tol=1e-06, lr=0.1, beta_1=0.9,
                          beta_2=0.99, noise_factor=1e-08, eps=1e-10, amsgrad=False, snapshot_dir=None)

    # Run optimization
    result = adam_optimizer.minimize(objective_function, initial_point, gradient_function)
    initial_point = result.x

In [5]:
# TSP- 4 distance matrix
D = np.array([[0, 2, 3],
              [2, 0, 4],
              [3, 4, 0]])


F = np.array([[0, 3, 4],
              [3, 0, 1],
              [4, 1, 0 ]])


'Affinity matrix'
K = TensorProduct(F, D)
print(K)

[[ 0  0  0  0  6  9  0  8 12]
 [ 0  0  0  6  0 12  8  0 16]
 [ 0  0  0  9 12  0 12 16  0]
 [ 0  6  9  0  0  0  0  2  3]
 [ 6  0 12  0  0  0  2  0  4]
 [ 9 12  0  0  0  0  3  4  0]
 [ 0  8 12  0  2  3  0  0  0]
 [ 8  0 16  2  0  4  0  0  0]
 [12 16  0  3  4  0  0  0  0]]
